In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import os
import re

import pickle

from tqdm import tqdm

In [2]:
path = os.path.join("..","data","./newdata_clean.xlsx")
n_cpu = 15
batch_size = 10000

loadpath = "processed_data_lda_wo_html"
#loadpath = "processed_data_not_rmsw"

## Read Data

In [3]:
df = pd.read_excel(path)

df = df.dropna() # drop nan entry
# df[pd.isnull(df).any(axis=1)]

le = preprocessing.LabelEncoder()
le.fit(df['catName'].unique())
num_classes = len(le.classes_)
class_list = list(le.classes_)

#print(class_list)

print("number of classes:",num_classes)
df.loc[:,'catName'] = le.transform(df.loc[:,'catName'])
data = df.question

number of classes: 64


In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

import langid

from nltk.corpus import stopwords

namelist = ["paul", "jim", "larry", "ken", "wright", "peter", "donna", "ian", "rick", "richard", "william", "john", "chris", "tony", "joseph"]
stw = stopwords.words('english') + ["nbsp", "would", "cant", "attach", "file", "hey", "quot", "dont", "cyberlink", "guy", "wont", "didnt", "doesnt"]
#+ ['nbsp', 'powerdirector', 'cyberlink', 'powerdvd', 'power', 'director', 'ba']
print(stw)
print("Stopwords length: {}".format(len(stw)))

def _filter(x):
    cut = 0
    redundant_pos = re.search(r"dear\s+valued\s+customer,", x)
    if redundant_pos != None:
        print("cut dear valued customer,")
        x = x[:redundant_pos.start()]
        cut = 1
    redundant_pos = re.search(r"order\s+confirmation\s+order\s+number:", x)
    if redundant_pos != None:
        print("cut order confirmation order number:")
        x = x[:redundant_pos.start()]
        cut = 1
    redundant_pos = re.search(r"<li>comment:</li>", x)
    if redundant_pos != None:
        print("cut <li>comment:</li>")
        x = x[redundant_pos.end():]
        cut = 1
    redundant_pos = re.search(r"time\s+of\s+this\s+report:", x)
    if redundant_pos != None:
        print("cut time of this report:")
        x = x[:redundant_pos.start()]
        cut = 1
    redundant_pos = re.search(r"this\s+correspondence\s+is\s+from", x)
    if redundant_pos != None:
        print("cut this correspondence is from")
        x = x[:redundant_pos.start()]
        cut = 1
    redundant_pos = re.search(r"forwarded\s+message", x)
    if redundant_pos != None:
        print("cut forwarded message")
        x = x[:redundant_pos.start()]
        cut = 1
    redundant_pos = re.search(r"(best)?(kind)?\s+regards", x)
    if redundant_pos != None:
        print("cut best|kind regards")
        x = x[:redundant_pos.start()]
        cut = 1
    redundant_pos = re.search(r"media\s+source\s+error\s+report", x)
    if redundant_pos != None:
        print("cut Media Source Error Report")
        x = x[:redundant_pos.start()]
        cut = 1
        
    x = re.sub(r'<[^<]*?/?>', ' ', x)                          # remove all html tag
    x = re.sub(r"\w{5}(-\w{5}){5}", "", x)                         # remove key tokex, ex: sd2kk-33j7v-na4m2-m8n5s-sjaxq-bramm
    x = re.sub(r'https?:\/\/[^ ]*', ' ', x)                        # remove all url
    x = re.sub(r'\S*@\S*\s?', ' ', x)                              # remove all email address
    x = re.sub(r'\S*\.\S*\s?', ' ', x, flags=re.IGNORECASE)        # remove all filename
    x = re.sub(r'[^a-z A-Z]', ' ', x)                              # remove all non-english alphabat
    x = re.sub(r"power\s+dvd", "powerdvd", x, flags=re.IGNORECASE)
    x = re.sub(r"power\s+director", "powerdirector", x, flags=re.IGNORECASE)
    x = re.sub(r"audio\s+director", "audiodirector", x, flags=re.IGNORECASE)
    x = re.sub(r"color\s+director ", "colordirector ", x, flags=re.IGNORECASE)
    x = re.sub(r"action\s+director", "actiondirector", x, flags=re.IGNORECASE)
    x = re.sub(r"makeup\s+director ", "makeupdirector ", x, flags=re.IGNORECASE)
    x = re.sub(r"director\s+suite", "directorsuite", x, flags=re.IGNORECASE)
    x = re.sub(r"audio\s+director", "audiodirector", x, flags=re.IGNORECASE)
    x = re.sub(r"photo\s+director", "photodirector ", x, flags=re.IGNORECASE)
    x = re.sub(r"blue?[-\s]*rays?", "bluray", x, flags=re.IGNORECASE)
    x = re.sub(r"power\s*(2|(to))\s*go", "power2go", x, flags=re.IGNORECASE)
    x = re.sub(r"cyber\s+link", "cyberlink", x, flags=re.IGNORECASE)
    x = re.sub(r"pdr", "powerdirector", x, flags=re.IGNORECASE)
    x = re.sub(r"pdvd", "powerdvd", x, flags=re.IGNORECASE)
    x = re.sub(r"pls", "please", x, flags=re.IGNORECASE)
    x = re.sub(r"add[-\s]*ons?", "addon", x, flags=re.IGNORECASE)
    x = re.sub(r"media[-\s]*suite", "mediasuite", x, flags=re.IGNORECASE)
    
    '''
    if cut:
        print(x)
    '''
    return x
'''
def _correct_word(text1):
    pattern = re.compile(r"(.)\1{2,}")
    text2 = pattern.sub(r"\1\1", text1) # reduce lengthening
    #if text1 != text2:
    #    print(text1, text2)
    text3 = spell(text2).lower() # spell correction
    #if text2 != text3:
    #    print(text2, text3)
    return text3
'''
def _get_wordnet_pos(tag):
    if tag =='J':
        return wordnet.ADJ
    elif tag =='V':
        return wordnet.VERB
    elif tag =='N':
        return wordnet.NOUN
    elif tag =='R':
        return wordnet.ADV
    else:
        return None

def _lemmatization(tokens):
    tagged_sent = pos_tag(tokens) # [('The', 'DT'), ('striped', 'JJ'), ('for', 'IN'), ('best', 'JJS')]
    ret = []
    for tag in tagged_sent:
        wordnet_pos = _get_wordnet_pos(tag[1][0]) or wordnet.NOUN
        ret.append(wnl.lemmatize(tag[0], pos=wordnet_pos))
        #print(tag[0],tag[1][0],wordnet_pos,ret[-1])
    return ret

def _remove_stopword(tokens):
    ret = []
    for word in tokens:
        if word in namelist:
            print("skip name {}".format(word))
            continue
        if word not in stw and len(word) > 2:
            ret.append(word)
    return ret

def preprocess(sentence):
    sentence = _filter(sentence.lower())
    tokens = nltk.word_tokenize(sentence)

    #tokens = [self._correct_word(word) for word in tokens] # spell correction
    tokens = _lemmatization(tokens) # lemmatization
    tokens = _remove_stopword(tokens) # remove stopwords
    s = " ".join(tokens)

    return s, tokens

def process_batch(batch):   
    clean_batch = []
    reduced_batch = []
    token_batch = []
    for ori_s in tqdm(batch):
        s = ori_s.lower()
        ret = langid.classify(s)
        if ret[0] != "en" and ret[1] < -100: # remove language other than english
            #print(ret)
            #print(s)
            continue

        processed, tokens = preprocess(s)
        if len(tokens) <= 20: # remove too short sentence
            #print(s,processed)
            continue
        #print(processed)
        #print(tokens)
        reduced_batch.append(ori_s)
        clean_batch.append(processed)
        token_batch.append(tokens)
    return clean_batch, reduced_batch, token_batch

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package punkt to
[nltk_data]     /home/student/05/b05505004/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/student/05/b05505004/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
clean_data = []
reduced_data = []
token_data = []

n_workers = n_cpu
from multiprocessing import Pool
ret = [None] * n_workers
n_data = len(data)
print("Origin Data length:",n_data)
with Pool(processes=n_workers) as pool:
    for i in range(n_workers):
        batch_start = (n_data // n_workers) * i 
        if i == n_workers - 1:
            batch_end = n_data
        else:
            batch_end = (n_data // n_workers) * (i + 1)
        batch = data[batch_start:batch_end]
        ret[i] = pool.apply_async(process_batch, [batch])
    pool.close()
    pool.join()

for result in ret:
    clean_batch, reduced_batch, token_batch = result.get()
    clean_data += clean_batch
    reduced_data += reduced_batch
    token_data += token_batch
print("done")

Origin Data length: 106478


  0%|          | 0/7106 [00:00<?, ?it/s]

cut best|kind regards


  0%|          | 1/7098 [00:05<10:20:36,  5.25s/it]

cut best|kind regards


...

cut best|kind regards


100%|█████████▉| 7094/7098 [18:27<00:00, 37.25it/s]

cut best|kind regards
cut best|kind regards


100%|██████████| 7098/7098 [18:27<00:00,  6.41it/s]


done


In [6]:
output = {
    "clean_data": clean_data,
    "reduced_data": reduced_data,
    "token_data": token_data
}
with open(loadpath, "wb") as f:
    pickle.dump(output, f)